In [1]:
import os

from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 13


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674
4,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-10-05T09:00:26-05:00 -> 1696514426000, Until: 2023-10-05T21:00:26-05:00 -> 1696557626000


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp')
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,
2023-10-05 09:00:30-05:00,-147.2,28.82,NaN,28.78,27.29,31.28,30.61,29.65,27.62,30.1,28.8,30.21,30.22
2023-10-05 09:01:00-05:00,-147.2,28.82,NaN,28.80,27.29,31.31,30.67,29.66,27.62,NaN,28.8,30.18,30.25
2023-10-05 09:01:30-05:00,-147.2,28.84,NaN,28.80,27.33,31.36,30.77,29.72,27.65,30.1,28.8,30.13,30.26
2023-10-05 09:02:00-05:00,-147.2,28.85,NaN,28.84,27.32,31.34,30.78,29.74,27.68,30.2,28.8,30.14,30.26
2023-10-05 09:02:30-05:00,-147.2,28.88,NaN,28.87,27.32,31.35,30.84,29.73,27.72,30.3,28.8,30.18,30.27


## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,1.370000e+03,1357.000000,1345.000000,1354.000000,1334.000000,1353.000000,1333.000000,1337.000000,1346.000000,1313.000000,1350.000000,1361.00000,1363.000000
mean,-1.472000e+02,31.855365,33.016639,30.810871,29.423988,33.435610,33.394291,34.159297,30.264658,32.967098,30.941778,33.21058,32.745620
std,5.686418e-14,1.417731,2.524827,1.108311,1.419709,2.187512,2.272787,2.928145,1.458552,2.503275,1.949546,1.47171,1.752429
min,-1.472000e+02,28.820000,28.930000,27.550000,25.370000,29.720000,28.000000,29.650000,27.420000,27.500000,25.900000,30.11000,26.640000
25%,-1.472000e+02,30.760000,30.720000,30.350000,28.470000,31.390000,31.420000,31.300000,28.917500,30.800000,29.200000,31.93000,31.390000
50%,-1.472000e+02,31.870000,32.800000,30.740000,28.940000,33.210000,33.180000,33.820000,30.720000,33.000000,30.600000,33.32000,32.690000
75%,-1.472000e+02,33.190000,35.240000,31.510000,30.447500,35.430000,35.800000,37.230000,31.500000,35.200000,32.500000,34.41000,34.390000
max,-1.472000e+02,34.200000,37.410000,33.080000,32.630000,37.800000,37.710000,39.080000,32.160000,38.000000,35.100000,35.65000,35.620000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
if not is_ok:
    print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")